## Script to plot results of heterochromatin localization study

In [77]:
from os import listdir
from os.path import isfile, join

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
# specify directory to analyze
myDir=r'S:\ActiveUsers\Kasia\2019_Liu_forMacro\20190426\20190506-MSL\test\results'

In [74]:
myFiles = [f for f in listdir(myDir) if ('nucleusSignal' in f)]

myData_cells=pd.DataFrame()
myData_cells["cell"]= myFiles

for f in myFiles:
    
    myData_nuc=pd.read_csv(join(myDir, f))
    myData_nuc['total']=myData_nuc.Area*myData_nuc.Mean;
    myData_nuc.set_axis(['Area_nuc', 'Mean_nuc', 'StdDev_nuc', 'Mode_nuc', 'Slice','total_nuc'], axis=1, inplace=True)
    
    myData_het=pd.read_csv(join(myDir, f.replace("nucleus", "hetChrom")))
    myData_het["total"]=myData_het.Area*myData_het.Mean;
    myData_het.set_axis(['Area_het', 'Mean_het', 'StdDev_het', 'Mode_het', 'Slice','total_het'], axis=1, inplace=True)
    
    
    # merge by Slice
    myData = pd.merge(myData_nuc, myData_het, on='Slice',how="outer")
    
    myData_eu=pd.read_csv(join(myDir, f.replace("nucleus", "euChrom")))
    myData_eu['total']=myData_eu.Area*myData_eu.Mean;
    myData_eu.set_axis(['Area_eu', 'Mean_eu', 'StdDev_eu', 'Mode_eu', 'Slice','total_eu'], axis=1, inplace=True)
    
    # merge by Slice
    myData = pd.merge(myData, myData_eu, on='Slice',how="outer")
    
    # sanity check
    myData["check"]=myData.Area_nuc-myData.Area_het-myData.Area_eu
    
    myData_cells.loc[myData_cells['cell'] == f,'volume_nuc']=np.sum(myData.Area_nuc)
    myData_cells.loc[myData_cells['cell'] == f,'volume_het']=np.sum(myData.Area_het)
    myData_cells.loc[myData_cells['cell'] == f,'volume_eu']=np.sum(myData.Area_eu)
    
    myData_cells.loc[myData_cells['cell'] == f,'signal_nuc']=np.sum(myData.total_nuc)
    myData_cells.loc[myData_cells['cell'] == f,'signal_het']=np.sum(myData.total_het)
    myData_cells.loc[myData_cells['cell'] == f,'signal_eu']=np.sum(myData.total_eu)
    
    myData_cells.loc[myData_cells['cell'] == f,'check']=np.sum(myData.check)

# add ratios of volume and signal
myData_cells['volume_het_ratio']=myData_cells.volume_het/myData_cells.volume_nuc
myData_cells['volume_eu_ratio']=myData_cells.volume_eu/myData_cells.volume_nuc

myData_cells['signal_het_ratio']=myData_cells.signal_het/myData_cells.signal_nuc
myData_cells['signal_eu_ratio']=myData_cells.signal_eu/myData_cells.signal_nuc

In [75]:
myData_cells.head()

,cell,volume_nuc,volume_het,volume_eu,signal_nuc,signal_het,signal_eu,check,volume_het_ratio,volume_eu_ratio,signal_het_ratio,signal_eu_ratio
0,10_L2.czi_nucleusSignal.csv,595544.0,16605.0,578939.0,2.223512e+08,8000026.571,2.143512e+08,0.0,0.027882,0.972118,0.035979,0.964021
1,10_E2_nucleusSignal.csv,564597.0,9928.0,554669.0,1.039293e+08,2090623.481,1.018386e+08,0.0,0.017584,0.982416,0.020116,0.979884


### Plot ratios

3407.372999999999